In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences

## Импорт БД

In [5]:
df = pd.read_csv(fr"C:\projects\Python\ML\DataBases\russian_toxic_comments.csv", sep=",")
df

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0


## Разделение данных 

In [6]:
test_df = df.sample(1000, random_state=0)
test_df

,comment,toxic
9470,"Светофоры работают на автоблокировке, там где ...",0.0
3683,Общая знакомая просто не попала в эту половину\n,0.0
4168,Откуда такой акцент на Москве? У нас между про...,0.0
9524,Спасибо за ответ! Очень подробно и доходчиво.\n,0.0
7994,"Ну вопрос то в том и состоит, как долго эта бе...",0.0
...,...,...
2476,Хохлушка же даже бесплатно Возьмет в рот твой ...,1.0
4070,"Как установил суд, банк занизил полную стоимос...",0.0
13142,Ставить что то на комплектные саморезы ? Серье...,0.0
7201,Ну если святую воду взбить с елеем то таки СОЖ...,0.0


In [7]:
df = df.drop(test_df.index)
df

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
...,...,...
14406,Потому что запад прошел эту хуйню еще пару сот...,0.0
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0


In [8]:
val_df = df.sample(500, random_state=0)
val_df

,comment,toxic
12569,Лучший камент в этой ветке!\n,1.0
11069,"За крыжопник и двор, аардом в упор.\n",1.0
4554,Так закат может длиться не одну сотню лет. Но ...,0.0
6646,"Это вообще нонсенс, что мы все со спокойной ду...",0.0
14383,Что. ты. несёшь. дегенерат?\n,1.0
...,...,...
13326,"Люди путешествующие и или пожившие еще где то,...",0.0
1360,И первый магазин из списка который следует пос...,0.0
12119,"Вот мне интересно, я ещё в 18 мимо армии проле...",0.0
6037,"больного синдромом патау Лскал ролик, в которо...",1.0


In [9]:
train_df = df.drop(val_df.index)
train_df

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
...,...,...
14406,Потому что запад прошел эту хуйню еще пару сот...,0.0
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0


## Обработка текста

In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['comment'].to_list())

In [11]:
train_sequences = tokenizer.texts_to_sequences(train_df['comment'].to_list())
test_sequences = tokenizer.texts_to_sequences(test_df['comment'].to_list())
val_sequences = tokenizer.texts_to_sequences(val_df['comment'].to_list())

In [12]:
maxlen = 300  # максимальная длина текста

padded_train_sequences = pad_sequences(train_sequences, maxlen=maxlen)
padded_test_sequences = pad_sequences(test_sequences, maxlen=maxlen)
padded_val_sequences = pad_sequences(val_sequences, maxlen=maxlen)

## Создание модели

In [13]:
output_dim = 100  # максимальное количество обрабатываемых данных

In [14]:
model = Sequential([
    layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=output_dim),
    layers.Bidirectional(layers.LSTM(32)),
    layers.Dense(1, activation='sigmoid')
])

In [15]:
model.compile(
    optimizer='Adam',
    loss='binary_crossentropy',
    metrics=[keras.metrics.BinaryAccuracy(),
             keras.metrics.Recall(),
             keras.metrics.Precision()]
)

In [16]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [17]:
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=0, min_delta=0.01)]

In [18]:
history = model.fit(
    padded_train_sequences,
    train_df['toxic'].to_numpy(),
    validation_data=(padded_val_sequences, val_df['toxic'].to_numpy()),
    batch_size=64,
    callbacks=callbacks,
    epochs=20
)

Epoch 1/20
202/202 ━━━━━━━━━━━━━━━━━━━━ 22s 97ms/step - binary_accuracy: 0.7008 - loss: 0.5755 - precision: 0.5879 - recall: 0.2447 - val_binary_accuracy: 0.8400 - val_loss: 0.3687 - val_precision: 0.7862 - val_recall: 0.6994
Epoch 2/20
202/202 ━━━━━━━━━━━━━━━━━━━━ 18s 90ms/step - binary_accuracy: 0.9489 - loss: 0.1712 - precision: 0.9273 - recall: 0.9240 - val_binary_accuracy: 0.8540 - val_loss: 0.3673 - val_precision: 0.8082 - val_recall: 0.7239
Epoch 3/20
202/202 ━━━━━━━━━━━━━━━━━━━━ 19s 92ms/step - binary_accuracy: 0.9859 - loss: 0.0543 - precision: 0.9747 - recall: 0.9834 - val_binary_accuracy: 0.8720 - val_loss: 0.4240 - val_precision: 0.8898 - val_recall: 0.6933
Epoch 4/20
202/202 ━━━━━━━━━━━━━━━━━━━━ 19s 94ms/step - binary_accuracy: 0.9953 - loss: 0.0244 - precision: 0.9909 - recall: 0.9953 - val_binary_accuracy: 0.8620 - val_loss: 0.4150 - val_precision: 0.8176 - val_recall: 0.7423


## Проверка обучения

In [19]:
model.evaluate(padded_test_sequences, test_df['toxic'].to_numpy())

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - binary_accuracy: 0.8649 - loss: 0.3728 - precision: 0.8100 - recall: 0.7410


[0.3889049291610718,
 0.8619999885559082,
 0.7337461113929749,
 0.8200691938400269]

## Тестирование модели

In [41]:
sms = ['Верблюдов-то за что? Дебилы, бл...',
       'Прогноз на сегодня - солнечно',
       'Я убью тебя!',
       'Я люблю тебя!',
       'Кодишь отвратительно как старый пердун!']

In [42]:
sms = tokenizer.texts_to_sequences(sms)

In [43]:
sms = pad_sequences(sms, maxlen=maxlen)

In [45]:
pred = model.predict(sms)

In [59]:
model.predict_proba(sms)

array([[0.58608317, 0.41391683],
       [0.48153243, 0.51846757],
       [0.57052224, 0.42947776],
       [0.64629528, 0.35370472],
       [0.4823608 , 0.5176392 ]])